**Vanessa Navarro Coronado e Iván Sánchez Castellanos**

# Práctica 2 - Clasificación supervisada en scikit-learn

Como siempre, lo primero que hacemos es importar los paquetes necesarios, e inicializar nuestra semilla para nuestros experimentos.

In [1]:
# Always load all scipy stack packages
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(color_codes=True)

In [2]:
# This code configures matplotlib for proper rendering
%matplotlib inline
mpl.rcParams["figure.figsize"] = "8, 4"
import warnings
warnings.simplefilter("ignore")

In [3]:
seed=6342
np.random.seed(6342)

A continuación, cargamos los datos de los datasets Pima y Wisconsin. Cada uno será separado en dos conjuntos, uno con los atributos y otro con la clase del dataset. Esto es necesario para que la libreria scikit learn pueda ser utilizada.

In [12]:
# Update the file path to fit your system
dfPima = pd.read_csv("../data/pima.csv", dtype={ "label": 'category'})
dfAttributesPima = dfPima.drop('label', 1)
dfLabelPima = dfPima['label']

dfWisc = pd.read_csv("../data/wisconsin.csv", dtype={ "label": 'category'})
dfAttributesWisc = dfWisc.drop('label', 1)
dfLabelWisc = dfWisc['label']

Una vez cargados los datasets, debemos separarlos en dos conjuntos de Train y Test para poder realizar nuestros experimentos.

In [13]:
# Divide into train/test split for our experiments
from sklearn.model_selection import train_test_split
train_attsPima, test_attsPima, train_labelPima, test_labelPima = train_test_split( 
    dfAttributesPima,
    dfLabelPima,
    test_size=0.2,
    random_state=seed,
    stratify=dfLabelPima)

train_attsWisc, test_attsWisc, train_labelWisc, test_labelWisc = train_test_split( 
    dfAttributesWisc,
    dfLabelWisc,
    test_size=0.2,
    random_state=seed,
    stratify=dfLabelWisc)

# KNN

In [17]:
from sklearn import neighbors

In [24]:
import sklearn.metrics as metrics

In [18]:
from sklearn.model_selection import GridSearchCV

In [27]:
clfPima = GridSearchCV(
    estimator = neighbors.KNeighborsClassifier(),
    param_grid = { 'n_neighbors' : [1,2,3,4,5] },
    scoring = 'accuracy',
    cv = 10
)

fittedPima = clfPima.fit(train_attsPima, train_labelPima)

In [28]:
fittedPima.best_params_

{'n_neighbors': 5}

In [29]:
# Get the mean score for each cv
meansPima = fitted.cv_results_['mean_test_score']
# Get the sd score for each cv
stdsPima = fitted.cv_results_['std_test_score']
# Get each specific configuration
confPima = fitted.cv_results_['params']

# Print the three things togheter
for mean, std, params in zip(meansPima, stdsPima, confPima):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))

0.707 (+/-0.128) for {'n_neighbors': 1}
0.717 (+/-0.049) for {'n_neighbors': 2}
0.721 (+/-0.067) for {'n_neighbors': 3}
0.730 (+/-0.096) for {'n_neighbors': 4}
0.733 (+/-0.068) for {'n_neighbors': 5}


In [30]:
predictionPima = clfPima.predict(test_attsPima)

In [31]:
metrics.confusion_matrix(test_labelPima, predictionPima)

array([[82, 18],
       [31, 23]])

In [32]:
metrics.accuracy_score(test_labelPima, predictionPima)

0.68181818181818177

In [33]:
# Recall
metrics.recall_score(test_labelPima, predictionPima, pos_label="tested_positive")

0.42592592592592593

In [34]:
# Precision
metrics.precision_score(test_labelPima, predictionPima, pos_label="tested_positive")

0.56097560975609762